In [9]:
import pandas as pd
import numpy as np
jeopardy = pd.read_csv('jeopardy.csv')
print(jeopardy.head())
print('\n', jeopardy.columns)

   Show Number    Air Date      Round                         Category  Value  \
0         4680  2004-12-31  Jeopardy!                          HISTORY   $200   
1         4680  2004-12-31  Jeopardy!  ESPN's TOP 10 ALL-TIME ATHLETES   $200   
2         4680  2004-12-31  Jeopardy!      EVERYBODY TALKS ABOUT IT...   $200   
3         4680  2004-12-31  Jeopardy!                 THE COMPANY LINE   $200   
4         4680  2004-12-31  Jeopardy!              EPITAPHS & TRIBUTES   $200   

                                            Question      Answer  
0  For the last 8 years of his life, Galileo was ...  Copernicus  
1  No. 2: 1912 Olympian; football star at Carlisl...  Jim Thorpe  
2  The city of Yuma in this state has a record av...     Arizona  
3  In 1963, live on "The Art Linkletter Show", th...  McDonald's  
4  Signer of the Dec. of Indep., framer of the Co...  John Adams  

 Index(['Show Number', ' Air Date', ' Round', ' Category', ' Value',
       ' Question', ' Answer'],
      dty

In [10]:
jeopardy.columns = jeopardy.columns.str.strip()
jeopardy.columns

Index(['Show Number', 'Air Date', 'Round', 'Category', 'Value', 'Question',
       'Answer'],
      dtype='object')

Normalizing text

In [11]:
import string
def norm(x):
    x = x.str.lower()
    x = [''.join(c for c in s if c not in string.punctuation) for s in x]
    x = [s for s in x if s]
    return x
jeopardy['clean_question'] = norm(jeopardy['Question'])
jeopardy['clean_answer'] = norm(jeopardy['Answer'])
jeopardy.head(1)

,Show Number,Air Date,Round,Category,Value,Question,Answer,clean_question,clean_answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus,for the last 8 years of his life galileo was u...,copernicus


Normalizing columns

In [12]:
def conv_str(ins):
    outs = ins.replace('$','').replace(',', '')
    if outs == 'None':
        return 0 
    else:
        outs = int(outs)
        return outs
jeopardy['clean_value'] = jeopardy['Value'].apply(conv_str)
jeopardy.head(1)

,Show Number,Air Date,Round,Category,Value,Question,Answer,clean_question,clean_answer,clean_value
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus,for the last 8 years of his life galileo was u...,copernicus,200


In [13]:
jeopardy['Air Date'] = pd.to_datetime(jeopardy['Air Date'])
jeopardy.head(1)

,Show Number,Air Date,Round,Category,Value,Question,Answer,clean_question,clean_answer,clean_value
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus,for the last 8 years of his life galileo was u...,copernicus,200


In [14]:
def sp_col(row):
    split_answer = row['clean_answer'].split(' ')
    
    split_question = row['clean_question'].split(' ')
    match_count = 0
    if 'the' in split_answer:
        split_answer.remove('the')
    if len(split_answer) == 0:
        return 0
    else:
        for item in split_answer:
            if item in split_question:
                match_count += 1
        return match_count/len(split_answer)
    
jeopardy['answer_in_question']=jeopardy.apply(sp_col, axis = 1)
jeopardy['answer_in_question'].mean()

0.06035277385469894

In [15]:
question_overlap = []
terms_used = set()
for index, row in jeopardy.iterrows():
    split_question = row['clean_question'].split(' ')
    templs = []
    for word in split_question:
        if len(word) >= 6:
            templs.append(word)
    split_question = templs
    match_count = 0
    for word in split_question:
        if word in terms_used:
            match_count += 1
        terms_used.add(word)
    if len(split_question) > 0:
        match_count = match_count / len(split_question)
    question_overlap.append(match_count)
jeopardy['question_overlap'] = question_overlap
jeopardy['question_overlap'].mean()

0.6919577992203644

nearly 70% words were reused in quetions

In [16]:
def val(row):
    if row['clean_value'] > 800:
        value = 1
    else:
        value = 0
    return value

jeopardy['high_value'] = jeopardy.apply(val, axis=1)

def high_low(word):
    low_count = 0
    high_count = 0
    for index, row in jeopardy.iterrows():
        split_row = row['clean_question'].split(' ')
        for word in split_row:
            if word in split_question:
                if row['high_value']==1:
                    high_count += 1
                else:
                    low_count += 1
    return high_count, low_count

observed_expected = []   
comparison_terms = list(terms_used)[:5]
for item in comparison_terms:
    ls = high_low(item)
    observed_expected.append(ls)
print(observed_expected)

['message', 'pushed', 'bullfights', 'reaction', 'sather']
[(160, 400), (160, 400), (160, 400), (160, 400), (160, 400)]


In [10]:
high_value_count = jeopardy[jeopardy['high_value']==1].shape[0]
low_value_count = jeopardy[jeopardy['high_value']==0].shape[0]
chi_squared =[]

from scipy.stats import chisquare
import numpy as np

for item in observed_expected:
    total = item[0]+item[1]
    total_prop = total / jeopardy.shape[0]
    high_counts = total_prop * high_value_count
    low_counts = total_prop * low_value_count
    
    observed = np.array([item[0], item[1]])
    expected = np.array([high_counts, low_counts])
    chi_square = chisquare(observed, expected)
    chi_squared.append()
chi_squared

/dataquest/system/env/python3/lib/python3.4/site-packages/scipy/stats/stats.py:4350: RuntimeWarning: invalid value encountered in true_divide
  terms = (f_obs - f_exp)**2 / f_exp


[]